In [ ]:
import numpy as np
import pandas as pd

def read_dataset(dataset):
    df = pd.read_csv(dataset)

    # 将十六进制数据转换为整数
    for column in ["ID", "DATA[0]", "DATA[1]", "DATA[2]", "DATA[3]", "DATA[4]", "DATA[5]", "DATA[6]", "DATA[7]"]:
        df[column] = df[column].apply(lambda x: int(x, 16))

    # 将 Flag 列的值转换为整数
    df["Flag"] = df["Flag"].replace({'T': 1, 'R': 0})

    print(dataset + " ok!")
    return df

df_DoS = read_dataset("new_DoS_dataset.csv")
df_Fuzzy = read_dataset("new_Fuzzy_dataset.csv")
df_Spoofing = read_dataset("Spoofing_dataset.csv")
df_Replaying = read_dataset("Replaying_dataset.csv")
df_Drop = read_dataset("Drop_dataset.csv")
df_Masquerade = read_dataset("Masquerade_dataset.csv")

In [ ]:
df_DoS_train = df_DoS.iloc[0 : round(df_DoS.shape[0] * 0.5)]
df_DoS_test = df_DoS.iloc[round(df_DoS.shape[0] * 0.5) : df_DoS.shape[0]]

df_Fuzzy_train = df_Fuzzy.iloc[0 : round(df_Fuzzy.shape[0] * 0.5)]
df_Fuzzy_test = df_Fuzzy.iloc[round(df_Fuzzy.shape[0] * 0.5) : df_Fuzzy.shape[0]]

df_Spoofing_train = df_Spoofing.iloc[0 : round(df_Spoofing.shape[0] * 0.5)]
df_Spoofing_test = df_Spoofing.iloc[round(df_Spoofing.shape[0] * 0.5) : df_Spoofing.shape[0]]

df_Masquerade_train = df_Masquerade.iloc[0 : round(df_Masquerade.shape[0] * 0.5)]
df_Masquerade_test = df_Masquerade.iloc[round(df_Masquerade.shape[0] * 0.5) : df_Masquerade.shape[0]]

In [ ]:
df_Attack_train = pd.concat([df_DoS_train, df_Fuzzy_train, df_Spoofing_train, df_Masquerade_train])
X_train = df_Attack_train[["ID", "DATA[0]", "DATA[1]", "DATA[2]", "DATA[3]", "DATA[4]", "DATA[5]", "DATA[6]", "DATA[7]"]]
y_train = df_Attack_train["Flag"]

X_test_DoS = df_DoS_test[["ID", "DATA[0]", "DATA[1]", "DATA[2]", "DATA[3]", "DATA[4]", "DATA[5]", "DATA[6]", "DATA[7]"]]
y_test_DoS = df_DoS_test["Flag"]

X_test_Fuzzy = df_Fuzzy_test[["ID", "DATA[0]", "DATA[1]", "DATA[2]", "DATA[3]", "DATA[4]", "DATA[5]", "DATA[6]", "DATA[7]"]]
y_test_Fuzzy = df_Fuzzy_test["Flag"]

X_test_Spoofing = df_Spoofing_test[["ID", "DATA[0]", "DATA[1]", "DATA[2]", "DATA[3]", "DATA[4]", "DATA[5]", "DATA[6]", "DATA[7]"]]
y_test_Spoofing = df_Spoofing_test["Flag"]

X_test_Replaying = df_Replaying[["ID", "DATA[0]", "DATA[1]", "DATA[2]", "DATA[3]", "DATA[4]", "DATA[5]", "DATA[6]", "DATA[7]"]]
y_test_Replaying = df_Replaying["Flag"]

X_test_Drop = df_Drop[["ID", "DATA[0]", "DATA[1]", "DATA[2]", "DATA[3]", "DATA[4]", "DATA[5]", "DATA[6]", "DATA[7]"]]
y_test_Drop = df_Drop["Flag"]

X_test_Masquerade = df_Masquerade_test[["ID", "DATA[0]", "DATA[1]", "DATA[2]", "DATA[3]", "DATA[4]", "DATA[5]", "DATA[6]", "DATA[7]"]]
y_test_Masquerade = df_Masquerade_test["Flag"]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import joblib
from sklearn.metrics import confusion_matrix

In [ ]:
# 创建并训练决策树模型
dt_model = DecisionTreeClassifier(random_state=52)
dt_model.fit(X_train, y_train)

joblib.dump(dt_model, 'dt_model.pkl')

In [ ]:
# 在测试集上进行预测
y_pred_DoS = dt_model.predict(X_test_DoS)
y_pred_Fuzzy = dt_model.predict(X_test_Fuzzy)
y_pred_Spoofing = dt_model.predict(X_test_Spoofing)
y_pred_Replaying = dt_model.predict(X_test_Replaying)
y_pred_Drop = dt_model.predict(X_test_Drop)
for i in range(len(y_test_Drop)):
    if y_test_Drop.iloc[i] == 1:
        y_pred_Drop[i] == 0
y_pred_Masquerade = dt_model.predict(X_test_Masquerade)

In [ ]:
attack_types = ["DoS Attack", "Fuzzy Attack", "Spoofing Attack", "Replay Attack", "Drop Attack", "Masquerade Attack"]
y_tests = [y_test_DoS, y_test_Fuzzy, y_test_Spoofing, y_test_Replaying, y_test_Drop, y_test_Masquerade]
y_preds = [y_pred_DoS, y_pred_Fuzzy, y_pred_Spoofing, y_pred_Replaying, y_pred_Drop, y_pred_Masquerade]

for attack_type, y_test, y_pred in zip(attack_types, y_tests, y_preds):
    print(attack_type + ": ")
    
    # 假设y_test是真实标签，y_pred是模型预测的标签
    conf_matrix = confusion_matrix(y_test, y_pred)
    TN, FP, FN, TP = conf_matrix.ravel()
    
    # 计算准确率
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    # 计算真正率（召回率）
    TPR = TP / (TP + FN)
    
    # 计算假正率
    FPR = FP / (FP + TN)

    print("accuracy = " + str(accuracy))
    print("TPR = " + str(TPR))
    print("FPR = " + str(FPR) + '\n')